# Search SWOT passes

In [ ]:
from search_swot import MapSelection

In [ ]:
MapSelection().display()

In [ ]:
from ipyleaflet import Map, DrawControl, Rectangle


m = Map(center=(50, 354), zoom=5)

draw_control = DrawControl()

draw_control.rectangle = {
    "pathOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

m.add(draw_control)

m

In [ ]:
m.layers

In [ ]:
for l in m.layers:
    if isinstance(l, Rectangle):
        m.remove_layer(l)

In [ ]:
draw_control.clear_polygons()

In [ ]:
m.controls

In [ ]:
from ipyleaflet import Map, DrawControl, LayerGroup, Rectangle
from ipywidgets import Output

# Création de la carte
m = Map(center=(50, 3.54), zoom=5)

# Contrôle de dessin
draw_control = DrawControl()

# Style du rectangle
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

# Groupe de calques pour les formes dessinées
drawn_shapes = LayerGroup()
m.add_layer(drawn_shapes)

# Gestion de l'événement de dessin
def handle_draw(target, action, geo_json):
    # Efface les anciens dessins
    drawn_shapes.clear_layers()

    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        bounds = geo_json['geometry']['coordinates'][0]
        # Convertit les coordonnées GeoJSON [lon, lat] en [(lat, lon)]
        latlng_bounds = [(lat, lon) for lon, lat in bounds[:2]]
        rectangle = Rectangle(bounds=latlng_bounds,
                              color="#fca45d",
                              fill_color="#fca45d",
                              fill_opacity=1.0)
        drawn_shapes.add_layer(rectangle)

draw_control.on_draw(handle_draw)

# Ajout du contrôle à la carte
m.add(draw_control)

m

In [ ]:
from ipyleaflet import Map, DrawControl, Rectangle
from ipywidgets import Output

# Crée la carte
m = Map(center=(50, 3.54), zoom=5)

# Crée le contrôle de dessin
draw_control = DrawControl()

# Applique le style du rectangle
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

# Variable pour suivre le rectangle actuellement affiché
current_rectangle = []

# Callback quand on dessine une forme
def handle_draw(target, action, geo_json):
    global current_rectangle

    # Supprime le rectangle précédent s'il existe
    if current_rectangle:
        m.remove_layer(current_rectangle[0])
        current_rectangle = []

    # Si on a bien dessiné un rectangle
    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        coords = geo_json['geometry']['coordinates'][0]
        # Conversion GeoJSON [lon, lat] -> (lat, lon)
        bounds = [(lat, lon) for lon, lat in coords[:2]]
        rectangle = Rectangle(bounds=bounds,
                              color="#fca45d",
                              fill_color="#fca45d",
                              fill_opacity=1.0)
        m.add_layer(rectangle)
        current_rectangle = [rectangle]

draw_control.on_draw(handle_draw)

# Ajoute le contrôle de dessin à la carte
m.add(draw_control)

m

In [ ]:
from ipyleaflet import Map, DrawControl, Rectangle
from ipywidgets import Output

# Créer la carte
m = Map(center=(50, 3.54), zoom=5)

# Créer le contrôle de dessin
draw_control = DrawControl()

# Appliquer le style du rectangle
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

# Suivi manuel du dernier calque ajouté
last_layer = {'layer': None}

# Fonction de callback pour le dessin
def handle_draw(target, action, geo_json):
    # Supprimer l'ancien calque s'il existe
    if last_layer['layer'] is not None:
        try:
            m.substitute_layer(last_layer['layer'], None)
        except Exception as e:
            print(f"Erreur lors de la suppression : {e}")
        last_layer['layer'] = None

    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        coords = geo_json['geometry']['coordinates'][0]
        bounds = [(lat, lon) for lon, lat in coords[:2]]
        rectangle = Rectangle(bounds=bounds,
                              color="#fca45d",
                              fill_color="#fca45d",
                              fill_opacity=1.0)
        m.add_layer(rectangle)
        last_layer['layer'] = rectangle

draw_control.on_draw(handle_draw)

# Ajouter le contrôle à la carte
m.add(draw_control)

m


In [ ]:
from ipyleaflet import Map, DrawControl, Rectangle

# Créer une carte centrée en Europe
m = Map(center=(50, 3.54), zoom=5)

# Ajouter un contrôle de dessin avec style personnalisé
draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

# Suivre le dernier rectangle ajouté
current_layer = {'rectangle': None}

# Fonction exécutée lorsqu'on dessine
def on_draw(target, action, geo_json):
    # Supprimer l'ancien rectangle s'il existe
    if current_layer['rectangle']:
        m.remove_layer(current_layer['rectangle'])

    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        coords = geo_json['geometry']['coordinates'][0]
        # GeoJSON = [lon, lat] → Leaflet = (lat, lon)
        bounds = [(lat, lon) for lon, lat in coords[:2]]
        rect = Rectangle(bounds=bounds,
                         color="#fca45d",
                         fill_color="#fca45d",
                         fill_opacity=1.0)
        m.add_layer(rect)
        current_layer['rectangle'] = rect

# Lier la fonction au contrôle
draw_control.on_draw(on_draw)

# Ajouter le contrôle à la carte
m.add(draw_control)

# Afficher la carte
m


In [ ]:
from ipyleaflet import Map, DrawControl
import threading
import time

# Créer la carte
m = Map(center=(50, 3.54), zoom=5)

# Créer le contrôle de dessin
draw_control = DrawControl()

# Style pour le rectangle
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

# Cette variable garde la dernière forme dessinée
latest_shape = {'geo_json': None}

# Fonction de "nettoyage" différé pour supprimer les anciennes formes
def delayed_clear_and_restore(new_geo_json):
    time.sleep(0.1)  # attendre un petit peu que la forme soit ajoutée
    draw_control.clear()  # efface toutes les formes (y compris la nouvelle)
    time.sleep(0.05)
    draw_control.draw(new_geo_json)  # réaffiche uniquement la dernière

# Callback appelé quand on dessine une forme
def handle_draw(target, action, geo_json):
    if action == 'created':
        latest_shape['geo_json'] = geo_json
        # Lancer un thread pour effacer puis redessiner
        threading.Thread(target=delayed_clear_and_restore, args=(geo_json,)).start()

draw_control.on_draw(handle_draw)

# Ajouter le contrôle à la carte
m.add(draw_control)

# Afficher la carte
m


In [ ]:
from ipyleaflet import Map, DrawControl, Rectangle

# Créer la carte
m = Map(center=(50, 3.54), zoom=5)

# Créer le contrôle de dessin
draw_control = DrawControl()

# Appliquer le style de dessin
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 1.0
    }
}

# Suivre la dernière forme ajoutée
last_shape = {'layer': None}

# Fonction appelée quand une forme est dessinée
def handle_draw(target, action, geo_json):
    if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
        
        draw_control.clear_polygons()
        draw_control.polygon = {}
        
        # Supprimer l'ancien rectangle s'il existe
        if last_shape['layer'] is not None:
            try:
                m.remove_layer(last_shape['layer'])
            except Exception:
                pass  # cas où le layer a déjà été supprimé

        # Extraire les coordonnées [lon, lat]
        coords = geo_json['geometry']['coordinates'][0]

        # Extraire les extrêmes lat/lon pour former un rectangle
        lats = [lat for lon, lat in coords]
        lons = [lon for lon, lat in coords]
        south_west = (min(lats), min(lons))
        north_east = (max(lats), max(lons))

        draw_control.rectangle = {
            "shapeOptions": {
                "color": "#3388ff",        # bordure bleue
                "weight": 4,               # épaisseur du trait
                "opacity": 1.0,            # opacité du trait
                "fill": True,
                "fillColor": "#3388ff",    # fond bleu
                "fillOpacity": 0.2         # transparence du fond
            }
        }
        
        # Créer un rectangle avec les coins opposés
        rect = Rectangle(bounds=(south_west, north_east),
                         color="#3388ff",
                         fill_color="#3388ff",
                         fill_opacity=0.2)

        # Ajouter le nouveau rectangle à la carte
        m.add_layer(rect)
        last_shape['layer'] = rect

# Lier le callback au contrôle
draw_control.on_draw(handle_draw)

# Ajouter le contrôle à la carte
m.add(draw_control)

# Afficher la carte
m


In [ ]:
from ipyleaflet import Map, DrawControl, Rectangle

# Créer la carte
m = Map(center=(20, 0), zoom=2)

# Créer le DrawControl
draw_control = DrawControl()

# Activer uniquement les rectangles
draw_control.circle = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.marker = {}

# Style identique à Leaflet par défaut
draw_control.rectangle = {
    "shapeOptions": {
        "color": "#3388ff",       # contour
        "weight": 4,
        "opacity": 1.0,
        "fill": True,
        "fillColor": "#3388ff",   # remplissage
        "fillOpacity": 0.2        # transparence
    }
}

# Dernier rectangle affiché
last_shape = {'layer': None}

# Callback sur dessin
def handle_draw(target, action, geo_json):
    if action == 'created':
        if last_shape['layer']:
            try:
                m.remove_layer(last_shape['layer'])
            except:
                pass

        draw_control.clear_polygons()
        
        coords = geo_json['geometry']['coordinates'][0]
        lats = [lat for lon, lat in coords]
        lons = [lon for lon, lat in coords]
        sw = (min(lats), min(lons))
        ne = (max(lats), max(lons))

        rect = Rectangle(bounds=(sw, ne),
                         color="#3388ff",
                         weight=4,
                         opacity=1.0,
                         fill_color="#3388ff",
                         fill_opacity=0.2)

        m.add_layer(rect)
        last_shape['layer'] = rect

draw_control.on_draw(handle_draw)
m.add(draw_control)
m

In [ ]:
from ipyleaflet import Map, DrawControl
import time
import threading

m = Map(center=(20, 0), zoom=2)

draw_control = DrawControl()

# Activer seulement les rectangles
draw_control.circle = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.marker = {}

# Style par défaut (bleu Leaflet)
draw_control.rectangle = {
    "shapeOptions": {
        "color": "#3388ff",
        "weight": 4,
        "opacity": 1.0,
        "fill": True,
        "fillColor": "#3388ff",
        "fillOpacity": 0.2
    }
}

def handle_draw(dc, action, geo_json):
    if action == "created":
        # Retarder la suppression pour laisser la forme s'ajouter

        threading.Thread(target=delayed_clear).start()

draw_control.on_draw(handle_draw)
m.add(draw_control)
m


In [ ]:
from ipyleaflet import Map, DrawControl
import ipywidgets as widgets
from IPython.display import display

m = Map(center=(20, 0), zoom=2)

draw_control = DrawControl()

# Désactiver tous les outils natifs
draw_control.circle = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.marker = {}
draw_control.rectangle = {}  # <-- on retire le bouton rectangle

m.add(draw_control)

# Créer un bouton pour activer le dessin rectangle
draw_button = widgets.ToggleButton(
    value=False,
    description='Draw Rectangle',
    tooltip='Cliquez pour dessiner un rectangle',
)

def on_draw_button_change(change):
    if change['new']:
        # Activer le mode dessin rectangle
        draw_control.mode = 'rectangle'
    else:
        # Désactiver le mode dessin
        draw_control.mode = None

draw_button.observe(on_draw_button_change, names='value')

display(draw_button)
display(m)
